In [1]:
from torch.functional import meshgrid, norm
from torch.nn.modules import normalization
import torch
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F


class Swish(torch.nn.Module):
    def forward(self, x):
        return x * torch.sigmoid(x)

class MLP(nn.Module):
    def __init__(self, input_size, output_size, hidden_size, hidden_layers=5):
        super().__init__()
#        self.normalization = nn.Parameter(torch.randn(1))
        layers = np.array([nn.Linear(hidden_size, hidden_size) for i in range(hidden_layers)])
        layers = np.insert(layers,np.arange(1,hidden_layers+1),Swish())
        self.output = nn.Sequential(
        nn.Linear(input_size, hidden_size), nn.LayerNorm(hidden_size), Swish(),
        *layers,
        nn.Linear(hidden_size, output_size),nn.Softplus())

    def compute_normalization(self, axis):
        n_dim = len(axis)
        grid = torch.from_numpy(np.array(np.meshgrid(*axis)).reshape(n_dim,-1).T).float()
        value = self.unnormalized_pass(grid)
        shape = []
        for i in range(n_dim):
            shape.append(len(axis[i]))
        norm = value.T.reshape(shape)
        for i in range(n_dim):
            norm = torch.trapz(norm,x=torch.from_numpy(axis[i]).float(),axis=0)
        return norm

    def unnormalized_pass(self, x):
        return self.output(x)

    def forward(self, x, axis):
        return self.output(x)/self.compute_normalization(axis)


n_dim = 2
model = MLP(n_dim, 1, 128, 5)
model.train().float()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)



data = np.load('../datasets/Combined_GWTC_m1m2chieffz.npz')

input_data = torch.from_numpy(np.array([data['m1'],data['m2']/data['m1']])).float().T
#input_data = torch.from_numpy(np.array([data['m1']])).float().T


n_epoch = 10000

n_grid = 100
print('hi')
m1_axis = 10**np.linspace(np.log10(0.5),np.log10(100),n_grid)
q_axis = np.linspace(0.05,1,n_grid)
axis = [m1_axis,q_axis]
grid_flatten = torch.from_numpy(np.array(np.meshgrid(*axis)).reshape(n_dim,-1).T).float()
print('hi')
value = model(grid_flatten,axis)
shape = []
for i in range(n_dim):
    shape.append(len(axis[i]))
print('hi')
norm = value.reshape(shape).T
grid = grid_flatten.reshape(*shape,n_dim).T
print('hi')
# def compute_normalization(axis,model):
#     grid = torch.from_numpy(np.array(np.meshgrid(*axis)).reshape(n_dim,-1).T).float().
#     value = model(grid)
#     shape = []
#     for i in range(n_dim):
#         shape.append(len(axis[i]))
#     norm = value.T.reshape(shape)
#     for i in range(n_dim):
#         norm = torch.trapz(norm,x=torch.from_numpy(axis[i]).float(),axis=0)
#     return norm

for i in range(n_epoch):
    # Write training loop for MLP
    print('hi')
    output = model(input_data,axis)
    print('hi11')
    geometric_term = torch.sum(torch.log(torch.mean(output,dim=0)),dim=0)
#    exponential_term = data['m1'].shape[0]*torch.log(model.normalization)-model.normalization
    print(i)
    loss = -geometric_term#(exponential_term+geometric_term)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    print(loss)

def poisson_likelihood(N,mu):
    return N*np.log(mu)-mu

/var/folders/d3/x840qlg17x1f92cnsrkq62fw0000gn/T/ipykernel_3236/1774771994.py:53: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matricesor `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at  /Users/distiller/project/pytorch/aten/src/ATen/native/TensorShape.cpp:2318.)
  input_data = torch.from_numpy(np.array([data['m1'],data['m2']/data['m1']])).float().T


hi
hi
hi
hi
hi
hi11
0
tensor([359.3786], grad_fn=<NegBackward0>)
hi
hi11
1
tensor([359.3684], grad_fn=<NegBackward0>)
hi
hi11
2
tensor([359.3583], grad_fn=<NegBackward0>)
hi
hi11
3
tensor([359.3483], grad_fn=<NegBackward0>)
hi
hi11
4
tensor([359.3382], grad_fn=<NegBackward0>)
hi
hi11
5
tensor([359.3280], grad_fn=<NegBackward0>)
hi
hi11
6
tensor([359.3175], grad_fn=<NegBackward0>)
hi
hi11
7
tensor([359.3068], grad_fn=<NegBackward0>)
hi
hi11
8
tensor([359.2957], grad_fn=<NegBackward0>)
hi
hi11
9
tensor([359.2842], grad_fn=<NegBackward0>)
hi
hi11
10
tensor([359.2721], grad_fn=<NegBackward0>)
hi
hi11
11
tensor([359.2595], grad_fn=<NegBackward0>)
hi
hi11
12
tensor([359.2461], grad_fn=<NegBackward0>)
hi
hi11
13
tensor([359.2320], grad_fn=<NegBackward0>)
hi
hi11
14
tensor([359.2169], grad_fn=<NegBackward0>)
hi
hi11
15
tensor([359.2009], grad_fn=<NegBackward0>)
hi
hi11
16
tensor([359.1838], grad_fn=<NegBackward0>)
hi
hi11
17
tensor([359.1655], grad_fn=<NegBackward0>)
hi
hi11
18
tensor([359.145

KeyboardInterrupt: 

In [2]:
grid = torch.from_numpy(np.array(np.meshgrid(*axis)).reshape(n_dim,-1).T).float()


